In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [3]:
# Load and clean data
data = pd.read_csv('imu_data.csv')
data = data.dropna()

# Extract features and labels
X = data[['Roll', 'Pitch']].values
y = data['Direction'].values

# Encode target labels (strings to integers)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Save the label mapping for STM32 decoding
with open('labels.txt', 'w') as f:
    for i, label in enumerate(label_encoder.classes_):
        f.write(f"{i}: {label}\n")

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [4]:
# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42
)

# Build the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(2,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.1)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.6371 - loss: 0.9929 - val_accuracy: 0.9155 - val_loss: 0.7362
Epoch 2/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8829 - loss: 0.6405 - val_accuracy: 0.9437 - val_loss: 0.4406
Epoch 3/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9564 - loss: 0.3913 - val_accuracy: 0.9859 - val_loss: 0.2223
Epoch 4/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9516 - loss: 0.2454 - val_accuracy: 0.9859 - val_loss: 0.1331
Epoch 5/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9489 - loss: 0.1726 - val_accuracy: 0.9859 - val_loss: 0.0914
Epoch 6/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9655 - loss: 0.1157 - val_accuracy: 1.0000 - val_loss: 0.0705
Epoch 7/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9706 - loss: 0.0885 - val_accuracy: 1.0000 - val_loss: 0.0608
Epoch 8/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9738 - loss: 0.0809 - val_accuracy: 1.0000 - val_lo

In [5]:
# Save Keras model
model.save('gesture_model.h5')

# Convert to TFLite (float32 input/output for STM32 X-CUBE-AI)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open('gesture_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("\n✅ TFLite model exported as 'gesture_model.tflite'")
print("📄 Label map saved to 'labels.txt'")


Saved artifact at '/tmp/tmp2jdeuzpd'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 2), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  133968692239568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133968692240912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133968692243024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133968692240528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133968692240336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133968692243216: TensorSpec(shape=(), dtype=tf.resource, name=None)

✅ TFLite model exported as 'gesture_model.tflite'
📄 Label map saved to 'labels.txt'
